# Feature Engineering

---

1. Import packages
2. Load data
3. Feature engineering

---

## 1. Import packages

In [1]:
import pandas as pd
import numpy as np
import os

---
## 2. Load data

In [2]:
df = pd.read_csv("D:\\INTERNSHIP\\clean_data_after_eda.csv")
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [3]:
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,0.000908,2.086294,99.530517,44.235794,2.086425,9.953056e+01,44.236702,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000,0


---

## 3. Feature engineering

### Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above. Use this code to re-create this feature and then think about ways to build on this feature to create features with a higher predictive power.

In [4]:
price_df = pd.read_csv('price_data.csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [5]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


Now it is time to get creative and to conduct some of your own feature engineering! Have fun with it, explore different ideas and try to create as many as yo can!

In [6]:
# Continue Feature Engineering

# 1. Remove Unnecessary Columns
# Check for columns with low variance or single unique values
low_variance_cols = [col for col in df.columns if df[col].nunique() <= 1]
print(f"Columns with low variance: {low_variance_cols}")
# No columns to drop based on single unique values in this case
# Keep 'id' for merging, but it won't be used in modeling

# 2. Expand Date Columns
# Calculate tenure (days since activation)
df['tenure_days'] = (pd.to_datetime('2016-01-01') - df['date_activ']).dt.days
# Contract duration
df['contract_duration_days'] = (df['date_end'] - df['date_activ']).dt.days
# Time since last product modification
df['days_since_modif_prod'] = (pd.to_datetime('2016-01-01') - df['date_modif_prod']).dt.days
# Time to renewal
df['days_to_renewal'] = (df['date_renewal'] - pd.to_datetime('2016-01-01')).dt.days

# Extract year and month from date_activ
df['activation_year'] = df['date_activ'].dt.year
df['activation_month'] = df['date_activ'].dt.month

# 3. Combine Columns for Better Features
# Consumption ratio: recent vs. annual
df['cons_ratio_last_month'] = df['cons_last_month'] / (df['cons_12m'] + 1e-6)  # Avoid division by zero
# Margin per unit consumption
df['margin_per_cons'] = df['net_margin'] / (df['cons_12m'] + 1e-6)
# Total consumption (electricity + gas)
df['total_cons_12m'] = df['cons_12m'] + df['cons_gas_12m']
# Price change impact: Interaction between price change and consumption
df['price_cons_interaction'] = df['var_6m_price_off_peak'] * df['cons_12m']

# 4. Enhance Price Features from price_df
# Calculate price volatility (standard deviation of off-peak prices)
price_volatility = price_df.groupby('id').agg({
    'price_off_peak_var': 'std',
    'price_off_peak_fix': 'std'
}).reset_index()
price_volatility.columns = ['id', 'price_off_peak_var_std', 'price_off_peak_fix_std']

# Average prices over the year
avg_prices = price_df.groupby('id').agg({
    'price_off_peak_var': 'mean',
    'price_peak_var': 'mean',
    'price_mid_peak_var': 'mean',
    'price_off_peak_fix': 'mean',
    'price_peak_fix': 'mean',
    'price_mid_peak_fix': 'mean'
}).reset_index()
avg_prices.columns = ['id', 'avg_price_off_peak_var', 'avg_price_peak_var', 'avg_price_mid_peak_var',
                     'avg_price_off_peak_fix', 'avg_price_peak_fix', 'avg_price_mid_peak_fix']

# Merge price features with main dataframe
df = df.merge(price_volatility, on='id', how='left')
df = df.merge(avg_prices, on='id', how='left')

# 5. Merge Off-Peak Difference Features
df = df.merge(diff, on='id', how='left')

# 6. Creative Features
# Consumption trend: Difference between recent and annual consumption
df['cons_trend'] = df['cons_last_month'] - (df['cons_12m'] / 12)  # Approximate monthly average
# Customer lifetime value proxy
df['clv_proxy'] = df['net_margin'] * df['num_years_antig'] * df['nb_prod_act']
# Contract renewal risk: Flag contracts ending soon (within 90 days)
df['contract_ending_soon'] = (df['date_end'] - pd.to_datetime('2016-01-01')).dt.days < 90
df['contract_ending_soon'] = df['contract_ending_soon'].astype(int)
# Recent product modification: Flag if modified in last 6 months
df['recent_modif_prod'] = df['days_since_modif_prod'] < 180
df['recent_modif_prod'] = df['recent_modif_prod'].astype(int)
# Price elasticity proxy: Change in consumption vs. price change
df['price_elasticity_proxy'] = df['cons_trend'] / (df['var_6m_price_off_peak'] + 1e-6)

# 7. Handle Missing Values
# Fill missing price volatility and average price features with median
price_cols = ['price_off_peak_var_std', 'price_off_peak_fix_std', 'avg_price_off_peak_var',
              'avg_price_peak_var', 'avg_price_mid_peak_var', 'avg_price_off_peak_fix',
              'avg_price_peak_fix', 'avg_price_mid_peak_fix', 'offpeak_diff_dec_january_energy',
              'offpeak_diff_dec_january_power']
for col in price_cols:
    df[col] = df[col].fillna(df[col].median())

# Replace infinite values with NaN and fill with median
df = df.replace([np.inf, -np.inf], np.nan)
for col in df.select_dtypes(include=['float64', 'int64']).columns:
    df[col] = df[col].fillna(df[col].median())

# 8. Display the new features
print("New features created:")
print(df.columns.tolist())
df.head()

# Save the enhanced dataset
df.to_csv('enhanced_data_with_features.csv', index=False)

Columns with low variance: []
New features created:
['id', 'channel_sales', 'cons_12m', 'cons_gas_12m', 'cons_last_month', 'date_activ', 'date_end', 'date_modif_prod', 'date_renewal', 'forecast_cons_12m', 'forecast_cons_year', 'forecast_discount_energy', 'forecast_meter_rent_12m', 'forecast_price_energy_off_peak', 'forecast_price_energy_peak', 'forecast_price_pow_off_peak', 'has_gas', 'imp_cons', 'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act', 'net_margin', 'num_years_antig', 'origin_up', 'pow_max', 'var_year_price_off_peak_var', 'var_year_price_peak_var', 'var_year_price_mid_peak_var', 'var_year_price_off_peak_fix', 'var_year_price_peak_fix', 'var_year_price_mid_peak_fix', 'var_year_price_off_peak', 'var_year_price_peak', 'var_year_price_mid_peak', 'var_6m_price_off_peak_var', 'var_6m_price_peak_var', 'var_6m_price_mid_peak_var', 'var_6m_price_off_peak_fix', 'var_6m_price_peak_fix', 'var_6m_price_mid_peak_fix', 'var_6m_price_off_peak', 'var_6m_price_peak', 'var_6m_price_m